# stimulus_properties.ipynb

In [1]:
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from tqdm.auto import tqdm

import metadata
from load import load_stimulus_movie
from metadata import STIMULUS_METADATA
from spectral_differentiation import join_axes, spectral_differentiation

## Load stimuli

In [2]:
stimuli = Parallel(n_jobs=len(STIMULUS_METADATA), verbose=5)(
    delayed(load_stimulus_movie)(stimulus) for stimulus in STIMULUS_METADATA.index
)
stimuli = dict(zip(STIMULUS_METADATA.index, stimuli))

[Parallel(n_jobs=14)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=14)]: Done   2 out of  14 | elapsed:    1.3s remaining:    8.0s
[Parallel(n_jobs=14)]: Done   5 out of  14 | elapsed:    3.3s remaining:    6.0s
[Parallel(n_jobs=14)]: Done   8 out of  14 | elapsed:    5.3s remaining:    3.9s
[Parallel(n_jobs=14)]: Done  11 out of  14 | elapsed:    7.3s remaining:    2.0s
[Parallel(n_jobs=14)]: Done  14 out of  14 | elapsed:    9.2s remaining:    0.0s
[Parallel(n_jobs=14)]: Done  14 out of  14 | elapsed:    9.2s finished


## Helper functions

### Differentiation

In [3]:
def reshape_movie(movie):
    """Reshape a stimulus from (time, x, y) to (trial, cell, sample), where each pixel is a 'cell'."""
    data = join_axes(1, 2, movie)
    # Move time to the last axis (assumes time is the first axis)
    data = np.moveaxis(data, 0, -1)
    return data


def compute_stimulus_differentiation(stimulus):
    data = reshape_movie(stimulus)
    distances = spectral_differentiation(
        data,
        sample_rate=metadata.TWOP_SAMPLE_RATE,
        window_length=1.0,
        metric='euclidean',
        log_frequency=False,
    )
    return np.median(distances)

### Energy

In [4]:
def spectral_energy_density(stimulus):
    # Remove DC component
    stimulus = stimulus - stimulus.mean()
    # Compute spectral energy density
    spectrum = np.fft.rfft(stimulus, axis=0)
    energy_spectral_density = np.abs(spectrum)**2
    return energy_spectral_density

## Compute

In [5]:
stimulus_properties = pd.DataFrame([
    {
        'stimulus': name,
        'mean luminance': stimulus.mean(),
        'contrast': stimulus.std(),
        'spectral energy': spectral_energy_density(stimulus).sum(),
        "stimulus differentiation": compute_stimulus_differentiation(stimulus),
    }
    for name, stimulus in tqdm(stimuli.items()) if stimulus is not None
])

  0%|          | 0/14 [00:00<?, ?it/s]

In [6]:
stimulus_properties['log(stimulus differentiation)'] = np.log10(stimulus_properties['stimulus differentiation'])

In [7]:
stimulus_properties.to_parquet('results/stimulus_properties.parquet')